In [2]:
from sagemaker import get_execution_role
import sagemaker
import boto3

session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session=session)
# #same function
# sess = sage.Session()

role = 'arn:aws:iam::002224604296:role/service-role/AmazonSageMaker-ExecutionRole-20200402T124851'
aws_region = session.region_name
print(f'AWS region:{aws_region}')

AWS region:us-east-1


In [4]:
model_path = "s3://{}/output".format(sagemaker.Session().default_bucket())
model_name = 'sagemaker-recsys-graph-inference'
s3_model_url = model_path+'/sagemaker-recsys-graph-2020-11-17-03-15-32-694/'
# #same function：
# model_path = "s3://{}/output".format(sagemaker_session.default_bucket())
print(model_path)

s3://sagemaker-us-east-1-002224604296/output


In [8]:
TrainJobName = 'sagemaker-recsys-graph-train'
training_image = '002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-train'
training_instance_type = 'ml.p2.xlarge'
training_params = {
    "RoleArn": role,
    "TrainingJobName": TrainJobName,
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "{}".format(training_instance_type),
        "VolumeSizeInGB": 1
    },
    "InputDataConfig": [
        {
            "ChannelName": "training",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "{}".format(s3_model_url),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "{}".format(s3_model_url)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

In [9]:
%%time

# sm_session = Session()
sm = boto3.client('sagemaker')
sm.create_training_job(**training_params)

# status = sm.describe_training_job(TrainingJobName=pipe_job)['TrainingJobStatus']
# print(status)

CPU times: user 20.7 ms, sys: 28 µs, total: 20.7 ms
Wall time: 224 ms


{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:002224604296:training-job/sagemaker-recsys-graph-train',
 'ResponseMetadata': {'RequestId': 'fd92ece1-2737-4f2e-9bba-2b53a8671383',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd92ece1-2737-4f2e-9bba-2b53a8671383',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Tue, 17 Nov 2020 12:42:30 GMT'},
  'RetryAttempts': 0}}

In [ ]:

serving_image = '002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-inference'
model_name = 'sagemaker-recsys-graph-inference'
serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model',
                     'RESNET_ARCH': 'resnet50' # 'resnet50' or 'resnet101'
                   }
}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

print(create_model_response)

In [ ]:
endpoint_config_name=f'{model_name}-endpoint-config'
print(endpoint_config_name)


In [ ]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name, 
    model_name=model_name, 
    initial_instance_count=1, 
    instance_type='ml.p2.xlarge')
print(epc)

In [ ]:
endpoint_name=f'{model_name}-endpoint'
print(endpoint_name)

In [ ]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=endpoint_config_name, wait=True)
print(ep)

In [ ]:
import base64
import json

data = {"instance":'中国银行'}
client = boto3.client('sagemaker-runtime')

body=json.dumps(data).encode('utf-8')

print(body)
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)

# print(response[''])
# body=response['Body'].read()
# msg=body.decode('utf-8')

In [ ]:
print(response['Body'].read())

In [2]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [1]:
import sagemaker
print(sagemaker.__version__)

2.16.3.post0


In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'ml.p2.xlarge'

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_name='sagemaker-recsys-graph-train:latest',
                      image_uri='002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-train',
                      hyperparameters=hyperparameters)

estimator.fit()

2020-11-25 07:47:07 Starting - Starting the training job...
2020-11-25 07:47:09 Starting - Launching requested ML instances......
2020-11-25 07:48:21 Starting - Preparing the instances for training.........
2020-11-25 07:49:55 Downloading - Downloading input data
2020-11-25 07:49:55 Training - Downloading the training image..................
2020-11-25 07:52:58 Training - Training image download completed. Training in progress.sys path is ['/opt/ml/code', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/src/fasthan', '/src/dglke/python', '/usr/lib/python3/dist-packages']
Using backend: pytorch
Reading train triples....
Finished. Read 3603022 train triples.
|Train|: 3603022
/usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 9.379 seconds
[proc 0][Train](2/200


2020-11-25 07:53:34 Uploading - Uploading generated training model[proc 0][Train](114/200) average pos_loss: 4.833424091339111
[proc 0][Train](114/200) average neg_loss: 0.9149004220962524
[proc 0][Train](114/200) average loss: 2.874162197113037
[proc 0][Train](114/200) average regularization: 4.641847044695169e-05
[proc 0][Train] 2 steps take 0.104 seconds
[proc 0]sample: 0.023, forward: 0.037, backward: 0.038, update: 0.006
[proc 0][Train](116/200) average pos_loss: 4.809475898742676
[proc 0][Train](116/200) average neg_loss: 0.9664234220981598
[proc 0][Train](116/200) average loss: 2.887949585914612
[proc 0][Train](116/200) average regularization: 4.208775862934999e-05
[proc 0][Train] 2 steps take 0.084 seconds
[proc 0]sample: 0.002, forward: 0.037, backward: 0.038, update: 0.006
[proc 0][Train](118/200) average pos_loss: 4.913357973098755
[proc 0][Train](118/200) average neg_loss: 1.0135390758514404
[proc 0][Train](118/200) average loss: 2.9634485244750977
[proc 0][Train](118/200)


2020-11-25 07:54:45 Completed - Training job completed


In [23]:
result = estimator.deploy(1, instance_type)

Attaching to tmp6azdd0q4_algo-1-xuics_1
algo-1-xuics_1  | Using backend: pytorch
algo-1-xuics_1  | sys path is ['/opt/ml/code', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/src/dglke/python', '/usr/lib/python3/dist-packages']
http://212.129.155.247/fasthan/fasthan_base.zip not found in cache, downloading to /tmp/tmpt0fe3eq_
100% 144M/144M [00:20<00:00, 6.89MB/s] 
algo-1-xuics_1  | Finish download from http://212.129.155.247/fasthan/fasthan_base.zip
algo-1-xuics_1  | Copy file to /root/.fastNLP/fasthan/fasthan_base
algo-1-xuics_1  | loading vocabulary file /root/.fastNLP/fasthan/fasthan_base/vocab.txt
algo-1-xuics_1  | Load pre-trained BERT parameters from file /root/.fastNLP/fasthan/fasthan_base/model.bin.
tmp6azdd0q4_algo-1-xuics_1 exited with code 0
Aborting on container exit...


RuntimeError: Giving up, endpoint didn't launch correctly

In [14]:
# 单独测试代码
!pip install -r requirements

pip 20.0.2 from /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pip (python 3.6)


In [ ]:
import kg
import encoding
kg = kg.Kg('kg')
encoder = encoding.encoding(kg)